In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

from itertools import combinations
from math import factorial, log, sqrt

# Задача 6

В десятичной записи числа π среди первых 10002 знаков после запятой цифры 0, 1, . . . , 9 встречаются соответственно 968, 1026, 1021, 974, 1014, 1046, 1021, 970, 948, 1014 раз. Можно ли при уровне значимости 0.05 считать эти цифры случайными? При каком уровне значимости эта гипотеза отвергается?

Будем решать задачу с помощью критерия на основе отношения правдоподобия. Цифры являются случайными, если их вероятности равны (те вероятность появления каждой цифры равна $\frac{1}{10}$). Поэтому рассмотрим следующие гипотезы:  
- $H_0$: $p_{i} = \frac{1}{10}$, $\forall i = 0, \dots, 9$
- $H_1$: $p_{i} = \frac{X_{i}}{n}$, где $X_i$ - количество появлений i цифры $\forall i = 0, \dots, 9$

In [ ]:
alpha = 0.05

In [3]:
counts = [968, 1026, 1021, 974, 1014, 1046, 1021, 970, 948, 1014]
n = 10002

In [4]:
lambda_ = 2 * (sum([count * (log(count / n) - log(0.1)) for count in counts]))
"lambda = {}".format(lambda_)

'lambda = 9.398472367756636'

In [5]:
chi_square = stats.chi2.ppf(1 - alpha, 9)
"chi_square = {}".format(chi_square)

'chi_square = 16.918977604620448'

$\lambda$ предельно распределена по закону $\chi_{9}^{2}$ и меньше критического значения хи-квадрат при уровне значимости 0.05 c 9 степенями свободы. Следовательно, гипотеза $H_0$ о том, что цифры являются случайными в записи числа $\pi$, принимается

Чтобы отклонить гипотезу, нужно, чтобы $\lambda > \chi_{9}^{2}$ при заданном уровне значимости $\alpha$. Найдём его.

In [6]:
for alpha_ in np.linspace(0.05, 0.5, 100000):
    chi_square = stats.chi2.ppf(1 - alpha_, 9)
    if lambda_ > chi_square:
        print(alpha_)
        break

0.40133651336513365


In [7]:
"Гипотеза отвергается при уровне значимости {0:.3f} и выше".format(alpha_)

'Гипотеза отвергается при уровне значимости 0.401 и выше'

# Задача 7

Предположим, что у нас есть 10 статей, написанных автором, скрывающемся под псевдонимом. Мы подозреваем, что эти статьи на самом деле написаны некоторым известным писателем. Чтобы проверить эту гипотезу, мы подсчитали доли четырехбуквенных слов в 8-и сочинениях подозреваемого нами автора: .224 .261 .216 .239 .229 .228 .234 .216  
В 10 сочинениях, опубликованных под псевдонимом, доли четырехбуквенных слов равны .207 .204 .195 .209 .201 .206 .223 .222 .219 .200  
• Используйте критерий Вальда. Найдите p-value и 95%-ый доверительный интервал для разницы средних
значений. Какой можно сделать вывод исходя из найденных значений?  
• Используйте критерий перестановок. Каково в этом случае значение p-value. Какой можно сделать вывод?

#### Критерий Вальда

In [8]:
alpha = 0.05

Сравним две гипотезы:
- $H_0$: $\theta = 0$  
- $H_1$: $\theta \neq 0$  
где $\theta$ - разница средних значений

In [9]:
X1 = [.224, .261, .216, .239, .229, .228, .234, .216]
X2 = [.207, .204, .195, .209, .201, .206, .223, .222, .219, .200]

In [10]:
#Средние и дисперсия для X1
mu1 = np.mean(X1)
se1 = sqrt(np.mean([(x - mu1) ** 2 for x in X1]))
"mu1 = {}, se1 = {}".format(mu1, se1)

'mu1 = 0.230875, se1 = 0.013623853162743647'

In [11]:
#Средние и дисперсия для X2
mu2 = np.mean(X2)
se2 = sqrt(np.mean([(x - mu2) ** 2 for x in X2]))
"mu1 = {}, se1 = {}".format(mu2, se2)

'mu1 = 0.20860000000000004, se1 = 0.009178235124467013'

In [12]:
theta_hat = mu1 - mu2
se = sqrt((se1 ** 2) / len(X1) + (se2 ** 2) / len(X2))
"theta_hat = {}, se = {}".format(theta_hat, se)

'theta_hat = 0.02227499999999996, se = 0.0056236262211317'

In [13]:
W = (theta_hat - 0) / se
"W = {}".format(W)

'W = 3.9609673765831'

In [14]:
z = stats.norm.ppf(1 - alpha / 2)
"z = {}".format(z)

'z = 1.959963984540054'

In [15]:
"95% доверительный интервал: ({}, {})".format(theta_hat - z * se, theta_hat + z * se)

'95% доверительный интервал: (0.011252895144066747, 0.03329710485593318)'

$p-value = 2Ф(-|W|)$

In [16]:
p_value = 2 * stats.norm.cdf(-abs(W))
"p-value = {0:.7f}".format(p_value)

'p-value = 0.0000746'

$|W| > z_{\frac{\alpha}{2}}$, $\theta_{0} = 0$ не принадлежит $95\%$ доверительному интервалу, $p-value$ очень мало (меньше $0.01$). Следовательно, гипотеза о том, что статьи принадлежат данному автору отклоняется. 

#### Критерий перестановок

In [17]:
len1 = len(X1)
len2 = len(X2)

X = X1 + X2
p_value = 0
for p in combinations(X, len1):
    mu1 = np.mean(p)
    new_Y = [x for x in X if x not in p]
    mu2 = np.mean(new_Y)
    if len(new_Y) == 9:
        new_Y.append(0.216)
    if mu1 - mu2 > theta_hat:
        p_value += factorial(8) * factorial(10)

In [18]:
p_value /= factorial(18)
"p-value = {}".format(p_value)

'p-value = 0.0006398829928241693'

При заданном $p-value$ < 0.01 можно отклонить гипотезу того, что статьи принадлежат данному автору

# Задача 8

Говорят, Джордж Р.Р. Мартин, автор цикла “Песнь Льда и Огня”, истребляет Старков: чаще “убивает” персонажей, относящихся к этому дому, чем персонажей других домов. В таблице 1 приведено количество персонажей, относящихся к тому или иному дому, упомянутых за первые 4 книги, а так же количество погибших персонажей.  

a) Предлагается протестировать отличие уровня смертности дома Старков от уровня смертности каждого из других домов на 5% уровне значимости. Необходимо привести значения оценок вероятностей смертельных исходов для всех домов, а также найти p-value для каждого из трех тестов.  
  
b) Требуется протестировать множественную гипотезу “смертность дома Старков не отличается от смертности ни одного из домов”. Для этого предлагается воспользоваться методом Бонферрони. Также требуется построить тест по методу Benjamini-Hochberg обеспечивающий FDR не больше 5%.

In [19]:
stark_count = 72
lannister_count = 49
greyjoy_count = 41
nights_watch_count = 105

In [20]:
stark_dead_count = 18
lannister_dead_count = 11
greyjoy_dead_count = 12
nights_watch_dead_count = 41

#### Пункт а)

In [21]:
print("Оценка вероятностей смертельных исходов:")
print("Stark: {}".format(stark_dead_count / stark_count))
print("Lannister: {}".format(lannister_dead_count / lannister_count))
print("Greyjoy: {}".format(greyjoy_dead_count / greyjoy_count))
print("Night's Watch: {}".format(nights_watch_dead_count / nights_watch_count))

Оценка вероятностей смертельных исходов:
Stark: 0.25
Lannister: 0.22448979591836735
Greyjoy: 0.2926829268292683
Night's Watch: 0.3904761904761905


In [22]:
def Vald_test(count1, n1, count2, n2):
    p1 = count1 / n1    
    se1 = sqrt(p1 * (1 - p1) / n1)
    
    p2 = count2 / n2
    se2 = sqrt(p2 * (1 - p2) / n2)
    
    delta = p1 - p2
    se = sqrt(se1 ** 2 + se2 ** 2)
    
    W = (delta - 0) / se
    
    return W, delta, se

In [23]:
w1, delta1, se1 = Vald_test(stark_dead_count, stark_count,
                            lannister_dead_count, lannister_count)
p_value1 = 2 * stats.norm.cdf(-abs(w1))

print("Сравнение с Ланнистерами")
print("95% доверительный интервал: ({}, {})".format(delta1 - z * se1, delta1 + z * se1))
print("p-value: {}".format(p_value1))

Сравнение с Ланнистерами
95% доверительный интервал: (-0.1282827639054105, 0.1793031720686758)
p-value: 0.7451005302348559


In [24]:
w2, delta2, se2 = Vald_test(stark_dead_count, stark_count,
                            greyjoy_dead_count, greyjoy_count)
p_value2 = 2 * stats.norm.cdf(-abs(w2))

print("Сравнение с Грейджоями")
print("95% доверительный интервал: ({}, {})".format(delta2 - z * se2, delta2 + z * se2))
print("p-value: {}".format(p_value2))

Сравнение с Грейджоями
95% доверительный интервал: (-0.21414814393227413, 0.12878229027373758)
p-value: 0.6256243102948869


In [25]:
w3, delta3, se3 = Vald_test(stark_dead_count, stark_count,
                            nights_watch_dead_count, nights_watch_count)
p_value3 = 2 * stats.norm.cdf(-abs(w3))

print("Сравнение с черным замком")
print("95% доверительный интервал: ({}, {})".format(delta3 - z * se3, delta3 + z * se3))
print("p-value: {}".format(p_value3))

Сравнение с черным замком
95% доверительный интервал: (-0.2772653406025526, -0.0036870403498283633)
p-value: 0.04413638743136098


Гипотезу о том, что Джордж Мартин убивает Старков равновероятно с членами других семей принимается для Ланнистеров и Грейджоев, тк в 95% доверительные интервалы входит значение $\theta=0$. Однако для черного замка значение $p-value < 0.05$, поэтому гипотеза данная гипотеза отклоняется.

#### Пункт б)

In [26]:
p_values = np.sort([p_value1, p_value2, p_value3])
p_values

array([0.04413639, 0.62562431, 0.74510053])

#### Поправка Бонферрони

Метод поправки Бонферрони утверждает, что для уменьшения ложноположительных результатов необходимо отклонить те гипотезы, для которых $p-value$ по критерию $p_{i}<\frac{\alpha}{m}$

In [27]:
for p in p_values:
    if p < alpha / len(p_values):
        print('Гипотеза отклоняется')
    else:
        print('Гипотеза принимается')

Гипотеза принимается
Гипотеза принимается
Гипотеза принимается


Все гипотезы по поправе Бонферрони принимаются с уровнем значимости $0.05$, то есть Мартин убивает членов всех домов равновероятно

#### Тест по методу Benjamini-Hochberg

In [28]:
for i, p in enumerate(p_values):
    if p >= (i + 1) * alpha / len(p_values):
        print('Принять все остальные гипотезы')
        break
    else:
        print('Гипотеза не принимается')

Принять все остальные гипотезы


Все гипотезы по методу Бенджамини — Хохберга принимаются, начиная с самого маленького $p-value$

# Задача 9

Девочка каждый будний день путешествует в метро от станции A до станции B. Со станции A составы идут в двух направлениях: до станции B и до станции C. Если приходит поезд до станции C, Девочке приходится дожидаться следующего поезда. Девочке кажется, что ей очень везёт с поездами до станции B и что поезда до станции B ходят чаще, чем поезда до станции C, поэтому на протяжении двух месяцев записывает, до какой станции идут поезда, которые она успела увидеть, спустившись на станцию (таблица trains.csv). Необходимо проверить, ходят ли поезда до станции B значимо чаще, чем до станции C.  
  
Обозначим реальную частоту поездов до станции B через p и будем считать, что поезда приходят случайно и независимо друг от друга. Необходимо:  
a) Построить тест на основе критерия отношения правдоподобий для различения гипотез $H_0 : p = p_0$ vs. $H_1 : p \neq p_0$.  
b) Пусть $p_0 = \frac{1}{2}$. Сравнить (как аналитически, так и экспериментально) полученный тест с тестом Вальда
для различения этих гипотез.  
  
Примечание. Аналитическое сравнение тестов подразумевает доказательство их (асимптотической) эквивалентности или неэквивалентности, где под эквивалентностью понимается идентичность выносимых тестами решений.

In [29]:
data = pd.read_csv('trains.csv')
data.head()

,train_to_B
0,1
1,0
2,0
3,1
4,1


In [30]:
count_to_B = sum(data['train_to_B'].values)
n = len(data['train_to_B'].values)
p_hat = count_to_B / n
count_to_B, n

(34, 46)

#### Пункт а)

In [31]:
def lr(p0):
    lambda_ = 2 * (count_to_B * (log(p_hat) - log(p0))
               + (n - count_to_B) * (log(1 - p_hat) - log(1 - p0)))
    return lambda_

In [32]:
chi_square = stats.chi2.ppf(1 - 0.05, 1)
"chi_square = {}".format(chi_square)

'chi_square = 3.841458820694124'

#### Пункт б)  

In [33]:
p0 = 1 / 2
lr_ = lr(p0)
"lr = {}".format(lr_)

'lr = 10.96480740332921'

Была проверена гипотеза о том, что $p = p_0 = \frac{1}{2}$. Данная гипотеза отклонена тестом на основе критерия отношения правдоподобия с уровнем значимости $0.05$.

In [34]:
se = sqrt(p_hat * (1 - p_hat) / n)
"p_hat = {}, se = {}".format(p_hat, se)

'p_hat = 0.7391304347826086, se = 0.06474307670904994'

In [35]:
W = (p_hat - p0) / se
W

3.6935290526466815

In [36]:
z = stats.norm.ppf(1 - 0.05 / 2)
"95% доверительный интервал: ({}, {})".format(p_hat - z * se, p_hat + z * se)

'95% доверительный интервал: (0.6122363361845568, 0.8660245333806605)'

In [37]:
p_value = 2 * stats.norm.cdf(-abs(W))
"p-value = {}".format(p_value)

'p-value = 0.00022116322931875268'

Гипотеза $H_{0}: p = p_0 = \frac{1}{2}$ отклонена тестом Вальда, так как $p-value < 0.01$

На практике эквивалентность тестов показана, так как они выносят одно и тоже решение: гипотеза $H_0$ отклоняется